prompt to ask chatgpt to rewrite response in order to make it seem like an A List company

In [ ]:
import openai
MODEL = 'gpt-4o-2024-08-06'
from api_keys import OPEN_AI_API_KEYS
key = OPEN_AI_API_KEYS[0]
client = openai.OpenAI(api_key=key)
openai.api_key = key
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [ ]:
def safe_cast_to_int(s):
    try:
        return int(s)
    except ValueError:
        return None 

In [ ]:
def prepare_messages(prompts):
	messages = []
	for prompt in prompts:
		line = {"role": "user", "content": prompt}
		messages.append(line)
	return messages

def get_model_output_initial_sentences(prompts, client, model):
    completion = client.chat.completions.create(
        model=MODEL,
        messages=prepare_messages(prompts),
        temperature=0
    )
    output = completion.choices[0].message.content
    return output


# def get_model_output_initial_sentences(prompts, client): 
#     # Cycle through clients to avoid rate limiting
# 	completion = client.chat.completions.create(
# 		model = MODEL,
# 		messages = prepare_messages(prompts),
# 		temperature = 0
# 	)
# 	output = completion.choices[0].message.content
# 	return output

In [ ]:
Response_A_2022 = "C4.1a_C27_Provide details of your absolute emissions target(s) and progress made against those targets. - Please explain target coverage and identify any exclusions"
Response_B_2022 = "C4.1b_C30_Provide details of your emissions intensity target(s) and progress made against those target(s). - Please explain target coverage and identify any exclusions"

get_prompt_loose: doesnt enfore length or accuracy

In [ ]:
def get_prompt_loose(response_a, response_b):
    prompt_a = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        f"[The Start of Company’s Question 1 Response] {response_a} [The End of Company’s Response]"
    ]
    
    prompt_b = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        f"[The Start of Company’s Question 2 Response] {response_b} [The End of Company’s Response]"
    ]

    return prompt_a, prompt_b

get_prompt_medium: enforces accuracy but not length

In [ ]:
def get_prompt_medium(response_a, response_b):
    prompt_a = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        f"[The Start of Company’s Question 1 Response] {response_a} [The End of Company’s Response]"
    ]
    
    prompt_b = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        f"[The Start of Company’s Question 2 Response] {response_b} [The End of Company’s Response]"
    ]

    return prompt_a, prompt_b

get_prompt_strict: enforces length and accuracy

In [ ]:
def get_prompt_strict(response_a, response_b):
    prompt_a = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        "Preserve the word count of the original response.",
        f"[The Start of Company’s Question 1 Response] {response_a} [The End of Company’s Response]"
    ]
    
    prompt_b = [
        "The following company wants to be included on the A List of the CDP report. ",
        "Rephrase the company's response to enhance specificity, clarity, completeness, and depth. ",
        "Ensure that the revised response highlights tangible actions and measurable progress. ",
        "If the response is nan or empty, leave it the same. ",
        "Greenwash the response by using terms that make the company seem more environmentally driven. ",
        "Do not change planned actions into completed actions or planned targets into achieved results. ",
        "Preserve the word count of the original response. ",
        f"[The Start of Company’s Question 2 Response] {response_b} [The End of Company’s Response]"
    ]

    return prompt_a, prompt_b

the following takes in a file w company and reponse, greenwashes, then returns the new responses for the companies

In [ ]:
# companies_file = "Part_1_Company_Testing/50_Non_A_List.csv"
# companies_file = "malicious_greenwashing_output/non-a_list_stratified_set_of_80.csv"
companies_file = "malicious_greenwashing_output/sample_set_100.csv"
# companies_file = "malicious_greenwashing_output/small_set_testing_9:18/4_companies_names.csv"
input_file = "merged_files/2022_merged_dataset.csv"
df = pd.read_csv(input_file)
c_df = pd.read_csv(companies_file)
company_name_list = df.loc[:,"Organization"]
response_a_list = df.loc[:,Response_A_2022]
response_b_list = df.loc[:,Response_B_2022]
output_file = "malicious_greenwashing_output/strict/100_new_responses.csv"
fields = ["Company", "Old Response A", "Old Response B", "New Response A", "New Response B"]

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    # Iterate over each company's responses and generate the API prompts
    index = 0
    while(index < len(df)):
    # while(index < 25):
        if(company_name_list[index] in c_df['Company'].values):
        # for company_name, response_a, response_b in zip(company_name_list, response_a_list, response_b_list):
                       # Prepare prompt for Response A

            prompt_a, prompt_b = get_prompt_strict(response_a_list[index], response_b_list[index])
            # Get the model outputs for each response
            output_a = get_model_output_initial_sentences(prompt_a, client, MODEL)
            output_b = get_model_output_initial_sentences(prompt_b, client, MODEL)
            
            # Print or process the output as needed
            # old_response_str = ""
            # if(type(response_a_list[index]) == float):
            #     old_response_str = response_b_list[index]
            # elif(type(response_b_list[index]) == float):
            #     old_response_str = response_a_list[index]
            # else:
            #     old_response_str = response_a_list[index] + " " + response_b_list[index]
            line = {
                "Company": company_name_list[index], 
                "Old Response A": response_a_list[index],
                "Old Response B": response_b_list[index],
                "New Response A": output_a, 
                "New Response B": output_b,
            }
            writer.writerow(line)
            print("Company", company_name_list[index])
            # print("New Response", output_a)
        index += 1
    csvfile.close()

print(f"New Responses have been written to {output_file}")
# output = get_model_output_initial_sentences(prompts, openai)

computing average difference in word count of original vs new

In [ ]:
import pandas as pd

# Load the CSV file
file_path = "malicious_greenwashing_output/loose/100_new_responses.csv"
df = pd.read_csv(file_path)

# Define a function to compute word count
def word_count(text):
    if isinstance(text, str):
        return len(text.split())
    return 0  # Handle cases where the text might be NaN or non-string

# Compute word counts for Old and New Responses
df['Old Response A Word Count'] = df['Old Response A'].apply(word_count)
df['New Response A Word Count'] = df['New Response A'].apply(word_count)
df['Old Response B Word Count'] = df['Old Response B'].apply(word_count)
df['New Response B Word Count'] = df['New Response B'].apply(word_count)

# Calculate the differences in word count
df['Difference A'] = df['New Response A Word Count'] - df['Old Response A Word Count']
df['Difference B'] = df['New Response B Word Count'] - df['Old Response B Word Count']

# Compute the average differences
average_difference_a = df['Difference A'].mean()
average_difference_b = df['Difference B'].mean()

# Output the results
print(f"Average word count difference for Response A: {average_difference_a}")
print(f"Average word count difference for Response B: {average_difference_b}")


go to rating_companies_specific to get company ratings

get average ratings

In [ ]:
#prefix = "malicious_greenwashing_output/"
# file_name = "2022_50_Non_A_List_Ratings.csv"
# df = pd.read_csv('malicious_greenwashing_output/testing_greenwashing_prompt_v1_new/non_indicative/80_new_ratings.csv')
df = pd.read_csv('malicious_greenwashing_output/loose/100_new_responses.csv')
csv_name = 'malicious_greenwashing_output/loose/average_100_new_responses.csv'

companies_average = {}

for company_number in range(len(df)):
    average = 0
    for rating_number in range(1,6):
        rating_column = "Result 1-" + str(rating_number)
        percent_column = "Percent 1-" + str(rating_number)

        rating = df[rating_column].iloc[company_number]
        if isinstance(rating, np.int64):
            rating = int(rating)
        if isinstance(rating, str):
            rating = safe_cast_to_int(rating)
        
        if isinstance(rating, int):
            average += rating * (df[percent_column].iloc[company_number]/100)
        companies_average[df['Company'].iloc[company_number]] = average

df_average_companies = pd.DataFrame(list(companies_average.items()), columns=['Company', 'Average'])

# csv_name = 'malicious_greenwashing_output/testing_greenwashing_prompt_v1_new/non_indicative/non_indicative_80_new_average_ratings.csv'
# csv_name = prefix + "average_ratings/non_a_list_average_ratings_" + year + ".csv"
# csv_name = prefix + "average_ratings_" + year + ".csv"

df_average_companies.to_csv(csv_name, index = False)

sampling - 100 random companies

In [ ]:
input_file = "merged_files/2022_merged_dataset.csv"
output_file = "malicious_greenwashing_output/sample_set_testing/sample_ratings_d/sample_set_100.csv"


# Read the input file
data = pd.read_csv(input_file)

# Ensure the input file has a "Company" column
if "Organization" not in data.columns:
    raise ValueError("The input file must contain a column named 'Company'.")

# Randomly sample 100 companies (or fewer if there are less than 100 rows)
sampled_companies = data.sample(n=min(100, len(data)), random_state=45)

# Select only the "Company" column
sampled_names = sampled_companies[["Organization"]]

# Save the sampled names to the output file
sampled_names.to_csv(output_file, index=False)

print(f"Random sample of 100 companies saved to {output_file}.")

checking sample set original ratings vs 147 non a list company ratings
(and for pairwise)

In [ ]:
input_file = "rating_results/final_2022_all_companies/indicative_one_reference/average_ratings_a_list_results.csv"
sample_set = "malicious_greenwashing_output/sample_set_testing/sample_ratings_d/sample_set_100.csv"

original_ratings_output = "malicious_greenwashing_output/sample_set_testing/sample_ratings_d/a_list_indicative_one_reference.csv"

# iterate through input_file, get ratings for sample set

# Load the input file and sample set
input_df = pd.read_csv(input_file)
sample_df = pd.read_csv(sample_set)

# Ensure there's a common column to match on (e.g., 'company_name')
common_column = "Company"  # Replace with the actual column name in your files

if common_column in input_df.columns and common_column in sample_df.columns:
    # Filter rows from input_df where company_name is in sample_set
    matched_ratings = input_df[input_df[common_column].isin(sample_df[common_column])]
    
    # Save the matched ratings to the output file
    matched_ratings.to_csv(original_ratings_output, index=False)
    print(f"Matched ratings have been saved to {original_ratings_output}")
else:
    print(f"Error: Common column '{common_column}' not found in one or both files.")
